In [194]:
import pandas as pd
import numpy as np

tv = pd.read_csv(r'../input/tv-series-dataset/series_data.csv')
#tv.drop_duplicates(subset = ['Series_Title'], keep = 'last', inplace = True)
tv.head(10)

,Poster_Link,Series_Title,Runtime_of_Series,Certificate,Runtime_of_Episodes,Genre,IMDB_Rating,Overview,Star1,Star2,Star3,Star4,No_of_Votes
0,https://m.media-amazon.com/images/M/MV5BYTRiND...,Game of Thrones,(2011–2019),A,57 min,"Action, Adventure, Drama",9.3,Nine noble families fight for control over the...,Emilia Clarke,Peter Dinklage,Kit Harington,Lena Headey,1773458
1,https://m.media-amazon.com/images/M/MV5BMjhiMz...,Breaking Bad,(2008–2013),18,49 min,"Crime, Drama, Thriller",9.5,A high school chemistry teacher diagnosed with...,Bryan Cranston,Aaron Paul,Anna Gunn,Betsy Brandt,1468887
2,https://m.media-amazon.com/images/M/MV5BMTc5Zm...,The Walking Dead,(2010– ),18+,44 min,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,Andrew Lincoln,Norman Reedus,Melissa McBride,Danai Gurira,854698
3,https://m.media-amazon.com/images/M/MV5BNDVkYj...,Friends,(1994–2004),13+,22 min,"Comedy, Romance",8.9,Follows the personal and professional lives of...,Jennifer Aniston,Courteney Cox,Lisa Kudrow,Matt LeBlanc,829816
4,https://m.media-amazon.com/images/M/MV5BMjEzMD...,Stranger Things,(2016– ),15,51 min,"Drama, Fantasy, Horror",8.7,"When a young boy disappears, his mother, a pol...",Millie Bobby Brown,Finn Wolfhard,Winona Ryder,David Harbour,824966
5,https://m.media-amazon.com/images/M/MV5BMWY3NT...,Sherlock,(2010–2017),UA,88 min,"Crime, Drama, Mystery",9.1,A modern update finds the famous sleuth and hi...,Benedict Cumberbatch,Martin Freeman,Una Stubbs,Rupert Graves,808717
6,https://m.media-amazon.com/images/M/MV5BY2FmZT...,The Big Bang Theory,(2007–2019),U,22 min,"Comedy, Romance",8.1,A woman who moves into an apartment across the...,Johnny Galecki,Jim Parsons,Kaley Cuoco,Simon Helberg,724187
7,https://m.media-amazon.com/images/M/MV5BMTM5Mj...,Dexter,(2006–2021),A,53 min,"Crime, Drama, Mystery",8.6,"By day, mild-mannered Dexter is a blood-spatte...",Michael C. Hall,Jennifer Carpenter,David Zayas,James Remar,647136
8,https://m.media-amazon.com/images/M/MV5BNjg1MD...,How I Met Your Mother,(2005–2014),15+,22 min,"Comedy, Romance",8.3,A father recounts to his children - through a ...,Josh Radnor,Jason Segel,Cobie Smulders,Neil Patrick Harris,603824
9,https://m.media-amazon.com/images/M/MV5BMTUwMG...,True Detective,(2014– ),A,55 min,"Crime, Drama, Mystery",9.0,Seasonal anthology series in which police inve...,Vince Vaughn,Colin Farrell,Rachel McAdams,Taylor Kitsch,500194


In [195]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = 'english')

vectorized = tfidf.fit_transform(tv['Genre'])

vectorized.shape

(2000, 28)

In [196]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

sim = linear_kernel(vectorized, vectorized)

indices = pd.Series(tv.index, index = tv['Series_Title']).drop_duplicates()

In [197]:
def get_recommendations(title, sim = sim):
    
    idx = indices[title][0]
    print(idx)
    
    sim_scores = list(enumerate(sim[idx]))
    
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
    
    s_df = pd.DataFrame(sim_scores)
    s_df.columns = ['seq', 'similarity']
    s_df.set_index('seq', inplace = True)
    #print(s_df)
    
    z = tv['IMDB_Rating'].iloc[s_df.index]
    
    z = z > 6.0
    #print(z)
    
    s_df = s_df[z]
    #print(s_df)
    
    sim_scores = s_df['similarity'][1 : 21]
    #print(sim_scores)
    
    series_indices = sim_scores.index
    series_indices = series_indices.drop(idx)
    #print(series_indices)
    
    return (tv['Series_Title'].iloc[series_indices], sim_scores)

In [198]:
get_recommendations('The Flash')

32


(14                        Vikings
 72         Agents of S.H.I.E.L.D.
 96           Battlestar Galactica
 138                     Supergirl
 161          Star Trek: Discovery
 169           Legends of Tomorrow
 183                   Killing Eve
 185                 Falling Skies
 192                 Stargate SG-1
 198                    Revolution
 261            Stargate: Atlantis
 293    Star Trek: Deep Space Nine
 334                    Enterprise
 360             Star Trek: Picard
 375        Xena: Warrior Princess
 379             Into the Badlands
 429                      Timeless
 466                      Farscape
 467          Legend of the Seeker
 Name: Series_Title, dtype: object,
 seq
 14     1.0
 32     1.0
 72     1.0
 96     1.0
 138    1.0
 161    1.0
 169    1.0
 183    1.0
 185    1.0
 192    1.0
 198    1.0
 261    1.0
 293    1.0
 334    1.0
 360    1.0
 375    1.0
 379    1.0
 429    1.0
 466    1.0
 467    1.0
 Name: similarity, dtype: float64)